# 📊 Synthetic Burnout Dataset

Este dataset fue generado automáticamente con una rutina en Python (`generate_synthetic_dataset.py`) para simular 1000 semanas laborales.

### 🔍 ¿Qué contiene?

- 10 variables semanales (features) extraídas del entorno laboral digital (mails, calendarios, documentos).
- 1 índice continuo (`burnout_index`) entre 1.0 y 10.0 que estima el nivel de agotamiento.
- 1 etiqueta categórica (`tipo_semana`) que clasifica la semana en 4 grupos:
  1. Semana saludable
  2. Carga aceptable
  3. Carga excesiva
  4. Agotamiento extremo

### 🧠 ¿Cómo fue generado?

- Cada registro simula una semana realista.
- Se asignan aleatoriamente 7 features dentro del rango típico del tipo de semana elegido, y 3 fuera de ese rango.
- Se calcula un `burnout_index` ponderado por importancia de cada variable.
- Si el índice final cae en el rango deseado, el registro se guarda. Si no, se reintenta.

Más detalles (rangos, pesos, reglas) están documentados en el archivo de diseño compartido con el equipo.

[Enlace al archivo de diseño compartido](https://docs.google.com/spreadsheets/d/1XGnYfmlciyIsoUXT2XwYzwOj1E_ziqLzvIS85oTRTDE/edit?gid=1806434176#gid=1806434176)

In [58]:
import pandas as pd

# Carga del dataset generado
df = pd.read_csv("raw_data/synthetic_burnout_dataset.csv")

# Visualizamos las primeras filas
df.head(30)

,num_events_outside_hours,num_meetings_no_breaks,meetings_weekend,total_meeting_hours,docs_created,emails_sent,avg_meeting_duration,num_events,emails_sent_out_of_hours,docs_edited,burnout_index,tipo_semana
0,5,5,2,16,7,30,66,2,13,18,7.35,3
1,1,3,1,10,3,46,57,34,4,5,4.93,2
2,2,3,1,10,4,27,31,14,1,7,3.65,2
3,0,1,0,9,1,24,83,24,0,0,2.62,1
4,1,0,0,20,7,12,16,6,0,2,2.73,1
5,0,1,6,3,2,49,0,21,1,0,2.87,1
6,8,10,3,30,14,31,93,14,6,4,7.89,4
7,4,8,1,33,11,47,87,4,20,18,7.83,4
8,2,2,1,13,2,10,46,15,5,7,3.73,2
9,4,3,1,14,6,17,48,11,5,0,4.74,2


In [59]:
df.shape  # cantidad de filas y columnas

df['tipo_semana'].value_counts()  # distribución por tipo

tipo_semana
3    250
2    250
1    250
4    250
Name: count, dtype: int64

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   num_events_outside_hours  1000 non-null   int64  
 1   num_meetings_no_breaks    1000 non-null   int64  
 2   meetings_weekend          1000 non-null   int64  
 3   total_meeting_hours       1000 non-null   int64  
 4   docs_created              1000 non-null   int64  
 5   emails_sent               1000 non-null   int64  
 6   avg_meeting_duration      1000 non-null   int64  
 7   num_events                1000 non-null   int64  
 8   emails_sent_out_of_hours  1000 non-null   int64  
 9   docs_edited               1000 non-null   int64  
 10  burnout_index             1000 non-null   float64
 11  tipo_semana               1000 non-null   int64  
dtypes: float64(1), int64(11)
memory usage: 93.9 KB


In [61]:
df.tail(200)

,num_events_outside_hours,num_meetings_no_breaks,meetings_weekend,total_meeting_hours,docs_created,emails_sent,avg_meeting_duration,num_events,emails_sent_out_of_hours,docs_edited,burnout_index,tipo_semana
800,1,0,0,11,0,6,85,36,0,0,2.53,1
801,5,5,2,17,4,27,63,29,6,6,6.15,3
802,2,4,4,15,9,22,91,19,8,14,5.54,3
803,0,0,0,23,0,1,64,23,0,0,2.18,1
804,4,5,1,18,7,28,38,38,8,8,6.36,3
...,...,...,...,...,...,...,...,...,...,...,...,...
995,3,5,1,13,3,19,49,2,3,15,4.54,2
996,3,4,1,12,3,11,56,34,4,1,4.92,2
997,2,3,1,8,0,15,53,14,4,22,4.04,2
998,4,3,1,11,3,19,35,27,4,6,4.85,2
